Downloaded Weber County Parcels from [here](https://opendata.gis.utah.gov/datasets/utah-weber-county-parcels-lir?geometry=-112.599%2C41.075%2C-111.315%2C41.436)

In [4]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

In [10]:
# output folder
outputs = '.\\outputs'

# store path to gdb
gdb = '.\\Utah_Weber_County_Parcels_LIR.gdb'

In [6]:
# Copy parcel data
parcels = os.path.join(gdb, 'Parcels_Weber_LIR')
parcels_copy = arcpy.FeatureClassToFeatureClass_conversion(parcels, gdb, 'parcels_temp')

In [10]:
# load gdb into pandas dataframe
sdf = pd.DataFrame.spatial.from_featureclass(parcels_copy)

In [19]:
# inspect the shape
sdf.shape

(107670, 28)

In [13]:
# preview last 5 records
sdf.tail(5)

,OBJECTID,COUNTY_NAME,COUNTY_ID,ASSESSOR_SRC,BOUNDARY_SRC,DISCLAIMER,CURRENT_ASOF,PARCEL_ID,SERIAL_NUM,PARCEL_ADD,PARCEL_CITY,TAXEXEMPT_TYPE,TAX_DISTRICT,TOTAL_MKT_VALUE,LAND_MKT_VALUE,PARCEL_ACRES,PROP_CLASS,PRIMARY_RES,HOUSE_CNT,SUBDIV_NAME,BLDG_SQFT,BLDG_SQFT_INFO,FLOORS_CNT,FLOORS_INFO,BUILT_YR,EFFBUILT_YR,CONST_MATERIAL,SHAPE
107665,107666,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430009,086430009,,,,0,0.0,0.0,0.0,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06347616599999, 41.191944920..."
107666,107667,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430003,086430003,,,,0,0.0,0.0,0.0,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06385715399995, 41.192322325..."
107667,107668,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430004,086430004,,,,0,0.0,0.0,0.0,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06377360599998, 41.192322016..."
107668,107669,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430002,086430002,,,,0,0.0,0.0,0.0,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06394070099998, 41.192322633..."
107669,107670,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430001,086430001,,,,0,0.0,0.0,0.0,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06402424999999, 41.192322941..."


In [14]:
# count each value in property class field
sdf["PROP_CLASS"].value_counts()

111- SNGL FAM RES              63569
                               23432
119- PUD                        6762
116- CONDO                      3788
500- COMMERCIAL                 3124
112- DUPLEX                     1530
200-INDUSTRIAL                   903
952- EXEMPT - COM                642
113- 3-4 UNITS                   629
121-OUT BLDGS                    546
117- RECREATIONAL                543
812- PROD FARM                   510
108- SFR on COMM Zone            330
911- RES LOT                     266
114- 5-9 UNITS                   216
951- EXEMPT - RES                164
115- 10+ UNITS                   150
901- VAC - RES - DEV             134
999- UNDEVELOPED VL              103
957-RELATED PARCEL                87
917- REC-VAC-LOT                  55
502-COMM LAND with SFR            48
922- PUD LOT                      44
919- PUD-COMMON                   31
118- MOBILE HOME                  25
508- Both RES & COMM IMPRVD       12
902- C/I VACANT                   11
5

In [26]:
# query for single family parcels
sf_boolean = sdf['PROP_CLASS']=='111- SNGL FAM RES'

# add a column to indicate if parcel is single family
sf_parcels = sdf[sf_boolean]
sdf['isSingleFamily'] = sf_boolean
sdf

,OBJECTID,COUNTY_NAME,COUNTY_ID,ASSESSOR_SRC,BOUNDARY_SRC,DISCLAIMER,CURRENT_ASOF,PARCEL_ID,SERIAL_NUM,PARCEL_ADD,PARCEL_CITY,TAXEXEMPT_TYPE,TAX_DISTRICT,TOTAL_MKT_VALUE,LAND_MKT_VALUE,PARCEL_ACRES,PROP_CLASS,PRIMARY_RES,HOUSE_CNT,SUBDIV_NAME,BLDG_SQFT,BLDG_SQFT_INFO,FLOORS_CNT,FLOORS_INFO,BUILT_YR,EFFBUILT_YR,CONST_MATERIAL,SHAPE,isSingleFamily
0,1,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,153040007,153040007,2494 S 4150 W,,,516,567000.0,101974.0,0.97,111- SNGL FAM RES,None,,Hunter Place,2114.0,None,1.0,None,2003,2011,Frame Masonry Veneer,"{""rings"": [[[-112.07847957699994, 41.221276599...",True
1,2,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,140840011,140840011,1538 27TH ST,Ogden,,25,251000.0,55782.0,0.20,111- SNGL FAM RES,None,,Denver Place Add,1038.0,None,1.0,None,1926,1975,Frame Masonry Veneer,"{""rings"": [[[-111.93875957899996, 41.216565272...",True
2,3,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,081570007,081570007,4737 S 3600 W,Roy,,40,287000.0,71205.0,0.20,111- SNGL FAM RES,None,,,1488.0,None,1.0,None,1981,2001,Frame Vinyl,"{""rings"": [[[-112.06733202899994, 41.177368593...",True
3,4,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,090750194,090750194,5234 S 4950 W,Hooper,,31,483000.0,125200.0,1.26,111- SNGL FAM RES,None,,,2474.0,None,2.0,None,1997,2007,Frame Stucco,"{""rings"": [[[-112.098193418, 41.16882466400005...",True
4,5,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,171430003,171430003,524 W 2200 N,Harrisville,,20,226000.0,44507.0,0.14,111- SNGL FAM RES,None,,,792.0,None,1.0,None,1986,2003,Frame Vinyl,"{""rings"": [[[-111.99104493199997, 41.298192436...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107665,107666,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430009,086430009,,,,0,0.0,0.0,0.00,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06347616599999, 41.191944920...",False
107666,107667,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430003,086430003,,,,0,0.0,0.0,0.00,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06385715399995, 41.192322325...",False
107667,107668,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430004,086430004,,,,0,0.0,0.0,0.00,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06377360599998, 41.192322016...",False
107668,107669,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430002,086430002,,,,0,0.0,0.0,0.00,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06394070099998, 41.192322633...",False


In [28]:
# create empty column for building type attribute
sdf['BUILDING_TYPE'] = ''
sdf

,OBJECTID,COUNTY_NAME,COUNTY_ID,ASSESSOR_SRC,BOUNDARY_SRC,DISCLAIMER,CURRENT_ASOF,PARCEL_ID,SERIAL_NUM,PARCEL_ADD,PARCEL_CITY,TAXEXEMPT_TYPE,TAX_DISTRICT,TOTAL_MKT_VALUE,LAND_MKT_VALUE,PARCEL_ACRES,PROP_CLASS,PRIMARY_RES,HOUSE_CNT,SUBDIV_NAME,BLDG_SQFT,BLDG_SQFT_INFO,FLOORS_CNT,FLOORS_INFO,BUILT_YR,EFFBUILT_YR,CONST_MATERIAL,SHAPE,isSingleFamily,BUILDING_TYPE
0,1,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,153040007,153040007,2494 S 4150 W,,,516,567000.0,101974.0,0.97,111- SNGL FAM RES,None,,Hunter Place,2114.0,None,1.0,None,2003,2011,Frame Masonry Veneer,"{""rings"": [[[-112.07847957699994, 41.221276599...",True,
1,2,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,140840011,140840011,1538 27TH ST,Ogden,,25,251000.0,55782.0,0.20,111- SNGL FAM RES,None,,Denver Place Add,1038.0,None,1.0,None,1926,1975,Frame Masonry Veneer,"{""rings"": [[[-111.93875957899996, 41.216565272...",True,
2,3,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,081570007,081570007,4737 S 3600 W,Roy,,40,287000.0,71205.0,0.20,111- SNGL FAM RES,None,,,1488.0,None,1.0,None,1981,2001,Frame Vinyl,"{""rings"": [[[-112.06733202899994, 41.177368593...",True,
3,4,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,090750194,090750194,5234 S 4950 W,Hooper,,31,483000.0,125200.0,1.26,111- SNGL FAM RES,None,,,2474.0,None,2.0,None,1997,2007,Frame Stucco,"{""rings"": [[[-112.098193418, 41.16882466400005...",True,
4,5,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,171430003,171430003,524 W 2200 N,Harrisville,,20,226000.0,44507.0,0.14,111- SNGL FAM RES,None,,,792.0,None,1.0,None,1986,2003,Frame Vinyl,"{""rings"": [[[-111.99104493199997, 41.298192436...",True,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107665,107666,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430009,086430009,,,,0,0.0,0.0,0.00,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06347616599999, 41.191944920...",False,
107666,107667,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430003,086430003,,,,0,0.0,0.0,0.00,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06385715399995, 41.192322325...",False,
107667,107668,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430004,086430004,,,,0,0.0,0.0,0.00,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06377360599998, 41.192322016...",False,
107668,107669,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430002,086430002,,,,0,0.0,0.0,0.00,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06394070099998, 41.192322633...",False,


In [36]:
# st building type to 1 where is single family equals True
sdf['BUILDING_TYPE'] = np.where(sdf['isSingleFamily'] == True, 1, "")
sdf[['isSingleFamily','BUILDING_TYPE']].head(10)

,isSingleFamily,BUILDING_TYPE
0,True,1
1,True,1
2,True,1
3,True,1
4,True,1
5,True,1
6,True,1
7,True,1
8,True,1
9,True,1


## Multifamily Units

In [11]:
# inputs
taz_shp = '.\\Inputs\\TAZ.shp'
parcels = '.\\Inputs\\Utah_Weber_County_Parcels_LIR.gdb\\Parcels_Weber_LIR'

In [13]:
# output gdb
gdb = arcpy.CreateFileGDB_management(".\\Outputs", "weber.gdb")

In [19]:
# select parcels within modeling area
parcels_layer = arcpy.MakeFeatureLayer_management(parcels, 'parcels') 
arcpy.SelectLayerByLocation_management(parcels_layer, 'HAVE_THEIR_CENTER_IN', taz_shp)
parcels_for_modeling = arcpy.FeatureClassToFeatureClass_conversion(parcels_layer, gdb, 'parcels_for_modeling')

# add rings and part count fields
arcpy.AddField_management(parcels_for_modeling, 'ORIG_PARTS', 'LONG')
arcpy.AddField_management(parcels_for_modeling, 'ORIG_RINGS', 'LONG')
arcpy.AddField_management(parcels_for_modeling, 'NOW_PARTS', 'LONG')
arcpy.AddField_management(parcels_for_modeling, 'NOW_RINGS', 'LONG')

# Use cursor to calculate parts and rings
fields = ["shape@", 'ORIG_PARTS', 'ORIG_RINGS', 'NOW_PARTS', 'NOW_RINGS']
with arcpy.da.UpdateCursor(parcels_for_modeling, fields) as cursor:
    for row in cursor:
        shape = row[0]
        parts = shape.partCount
        rings = shape.boundary().partCount
        
        row[1] = parts
        row[2] = rings
        row[3] = parts
        row[4] = rings
        
        cursor.updateRow(row)

In [27]:
# filter for common areas, some small office parks and a few other strange cases may be present
parcels_for_modeling_layer = arcpy.MakeFeatureLayer_management(parcels_for_modeling, 'parcels_for_modeling_layer')
query = """ TAXEXEMPT_TYPE = '700-Common Area' """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
common_areas =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, 'common_areas')

In [28]:
# get condos and puds
query = """ PROP_CLASS IN ('116- CONDO', '119- PUD' ) AND TAXEXEMPT_TYPE <> '700-Common Area' """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
condos_and_puds =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, 'condos_and_puds')